# Implementing a Route Planner
In this project I used A* search to implement a "Google-maps" style route planning algorithm.

## The Map

In [10]:
from helpers import Map, load_map_10, load_map_40, show_map
import math
import chart_studio

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Map Basics

In [11]:
map_10 = load_map_10()
show_map(map_10)

AttributeError: 'Graph' object has no attribute 'node'

The map above (run the code cell if you don't see it) shows a disconnected network of 10 intersections. The two intersections on the left are connected to each other but they are not connected to the rest of the road network. This map is quite literal in its expression of distance and connectivity. On the graph above, the edge between 2 nodes(intersections) represents a literal straight road not just an abstract connection of 2 cities.

These `Map` objects have two properties you will want to use to implement A\* search: `intersections` and `roads`

**Intersections**

The `intersections` are represented as a dictionary. 

In this example, there are 10 intersections, each identified by an x,y coordinate. The coordinates are listed below. You can hover over each dot in the map above to see the intersection number.

In [5]:
map_10.intersections

{0: (0.7798606835438107, 0.6922727646627362),
 1: (0.7647837074641568, 0.3252670836724646),
 2: (0.7155217893995438, 0.20026498027300055),
 3: (0.7076566826610747, 0.3278339270610988),
 4: (0.8325506249953353, 0.02310946309985762),
 5: (0.49016747075266875, 0.5464878695400415),
 6: (0.8820353070895344, 0.6791919587749445),
 7: (0.46247219371675075, 0.6258061621642713),
 8: (0.11622158839385677, 0.11236327488812581),
 9: (0.1285377678230034, 0.3285840695698353)}

**Roads**

The `roads` property is a list where `roads[i]` contains a list of the intersections that intersection `i` connects to.

In [6]:
# this shows that intersection 0 connects to intersections 7, 6, and 5
map_10.roads[0] 

[7, 6, 5]

In [7]:
# This shows the full connectivity of the map
map_10.roads

[[7, 6, 5],
 [4, 3, 2],
 [1, 4, 3],
 [1, 2, 5, 4],
 [1, 2, 3],
 [0, 3, 7],
 [0],
 [0, 5],
 [9],
 [8]]

In [8]:
# map_40 is a bigger map than map_10
map_40 = load_map_40()
show_map(map_40)

AttributeError: 'Graph' object has no attribute 'node'

### Advanced Visualizations

The map above shows a network of roads which spans 40 different intersections (labeled 0 through 39). 

The `show_map` function which generated this map also takes a few optional parameters which might be useful for visualizing the output of the search algorithm you will write.

* `start` - The "start" node for the search algorithm.
* `goal`  - The "goal" node.
* `path`  - An array of integers which corresponds to a valid sequence of intersection visits on the map.

In [12]:
# run this code, note the effect of including the optional
# parameters in the function call.
show_map(map_40, start=5, goal=34, path=[5,16,37,12,34])

AttributeError: 'Graph' object has no attribute 'node'

### PathPlanner class

`__init__` - I initialize path planner with a map, M, and typically a start and goal node. If either of these are `None`, the rest of the variables here are also set to none. If you don't have both a start and a goal, there's no path to plan! 

- `closedSet` includes any explored/visited nodes. 
- `openSet` are any nodes on our frontier for potential future exploration. 
- `cameFrom` will hold the previous node that best reaches a given node
- `gScore` is the `g` in our `f = g + h` equation, or the actual cost to reach our current node
- `fScore` is the combination of `g` and `h`, i.e. the `gScore` plus a heuristic; total cost to reach the goal
- `path` comes from the `run_search` function, which is already built for you.

In [15]:
class PathPlanner():
    """Construct a PathPlanner Object"""
    def __init__(self, M, start=None, goal=None):
        """ """
        self.map = M
        self.start= start
        self.goal = goal
        self.closedSet = self.create_closedSet() if goal != None and start != None else None
        self.openSet = self.create_openSet() if goal != None and start != None else None
        self.cameFrom = self.create_cameFrom() if goal != None and start != None else None
        self.gScore = self.create_gScore() if goal != None and start != None else None
        self.fScore = self.create_fScore() if goal != None and start != None else None
        self.path = self.run_search() if self.map and self.start != None and self.goal != None else None
    
    def reconstruct_path(self, current):
        """ Reconstructs path after search """
        total_path = [current]
        while current in self.cameFrom.keys():
            current = self.cameFrom[current]
            total_path.append(current)
        return total_path
    
    def _reset(self):
        """Private method used to reset the closedSet, openSet, cameFrom, gScore, fScore, and path attributes"""
        self.closedSet = None
        self.openSet = None
        self.cameFrom = None
        self.gScore = None
        self.fScore = None
        self.path = self.run_search() if self.map and self.start and self.goal else None

    def run_search(self):
        """ """
        if self.map == None:
            raise(ValueError, "Must create map before running search. Try running PathPlanner.set_map(start_node)")
        if self.goal == None:
            raise(ValueError, "Must create goal node before running search. Try running PathPlanner.set_goal(start_node)")
        if self.start == None:
            raise(ValueError, "Must create start node before running search. Try running PathPlanner.set_start(start_node)")

        self.closedSet = self.closedSet if self.closedSet != None else self.create_closedSet()
        self.openSet = self.openSet if self.openSet != None else  self.create_openSet()
        self.cameFrom = self.cameFrom if self.cameFrom != None else  self.create_cameFrom()
        self.gScore = self.gScore if self.gScore != None else  self.create_gScore()
        self.fScore = self.fScore if self.fScore != None else  self.create_fScore()

        while not self.is_open_empty():
            current = self.get_current_node()

            if current == self.goal:
                self.path = [x for x in reversed(self.reconstruct_path(current))]
                return self.path
            else:
                self.openSet.remove(current)
                self.closedSet.add(current)

            for neighbor in self.get_neighbors(current):
                if neighbor in self.closedSet:
                    continue    # Ignore the neighbor which is already evaluated.

                if not neighbor in self.openSet:    # Discover a new node
                    self.openSet.add(neighbor)
                
                # The distance from start to a neighbor
                #the "dist_between" function may vary as per the solution requirements.
                if self.get_tentative_gScore(current, neighbor) >= self.get_gScore(neighbor):
                    continue        # This is not a better path.

                # This path is the best until now. Record it!
                self.record_best_path_to(current, neighbor)
        print("No Path Found")
        self.path = None
        return False

### Data Structures

In [16]:
def create_closedSet(self):
    """ Creates and returns a data structure suitable to hold the set of nodes already evaluated"""
    # Returns a set data structure
    explored = set()
    return explored

In [17]:
def create_openSet(self):
    """ Creates and returns a set data structure."""
    if self.start != None:
        frontier = set()
        frontier.add(self.start)
        
        return frontier

In [18]:
def create_cameFrom(self):
    """Creates and returns a dictionary data structure that shows which node can most efficiently be reached from another,
    for each node."""

    a = {}
    
    return a
    
    
    
    

In [23]:
def create_gScore(self):
    
    # Returns a dictionary that holds the cost of getting from the start node to that node, for each node.
    # for each node. The cost of going from start to start is zero. The rest of the node's values are
    # set to infinity.
    
    list_score = {}
    for i in range(len(self.map.roads)):
        list_score[i] = float('inf') 
    
    list_score[self.start] = float(0)
    
    return list_score
    
    
    

In [24]:
def create_fScore(self):
    """Creates and returns a data structure that holds the total cost of getting from the start node to the goal
    by passing by that node, for each node. That value is partly known, partly heuristic.
    For the first node, that value is completely heuristic."""
    # Returns a dictionary that holds the total cost of getting from the start node to the goal
    # by passing by that node, for each node. That value is partly known, partly heuristic.
    # For the first node, that value is completely heuristic. The rest of the node's value are 
    # set to infinity.
    
    list_score = {}
    for i in range(len(self.map.roads)):
        list_score[i] = float('inf')
    
    list_score[self.start] = heuristic_cost_estimate(self,self.start)
    
    return list_score 
    


### Set certain variables

The below functions help set certain variables if they weren't a part of initializating our `PathPlanner` class, or if they need to be changed for anothe reason.

In [25]:
def set_map(self, M):
    """Method used to set map attribute """
    self._reset()
    self.start = None
    self.goal = None
    # set a map value
    self.map = M



In [26]:
def set_start(self, start):
    """Method used to set start attribute """
    # Set start value. Removes the following goal, closedSet, openSet, cameFrom, gScore, fScore, 
    # and path attributes' values.
    self._reset(self)
    self.start = start
    self.goal = None

In [27]:
def set_goal(self, goal):
    """Method used to set goal attribute """
    self._reset(self)
    # Set goal value. 
    self.goal = goal

### Get node information

The below functions gets certain node information. In `is_open_empty`, I am checking whether there are still nodes on the frontier to explore.

In [37]:
def is_open_empty(self):
    """returns True if the open set is empty. False otherwise. """

    return set() == self.openSet


In [38]:
def get_current_node(self):
    """ Returns the node in the open set with the lowest value of f(node)."""

    temp={}
    for node in self.openSet:
        temp[node]=self.calculate_fscore(node)
    return min(temp, key=temp.get)


In [39]:
def get_neighbors(self, node):
    """Returns the neighbors of a node"""

    return self.map.roads[node]


### Scores and Costs

Main part of the calculation for determining the best path - calculating the various parts of the `fScore`.

In [40]:
def get_gScore(self, node):
    """Returns the g Score of a node"""
    
    return self.gScore[node]


In [41]:
def distance(self, node_1, node_2):
    """ Computes the Euclidean L2 Distance"""

    x_axis = self.map.intersections[node_2][0] - self.map.intersections[node_1][0]
    y_axis = self.map.intersections[node_2][1] - self.map.intersections[node_1][1]
    L2 = math.sqrt(x_axis**2 + y_axis**2)
    return L2

In [42]:
def get_tentative_gScore(self, current, neighbor):
    """Returns the tentative g Score of a node"""
    
    # Return the g Score of the current node plus distance from the current node to it's neighbors
    g_score = get_gScore(self,current)
    L2 = distance(self,current,neighbor)
    
    return g_score + L2


In [43]:
def heuristic_cost_estimate(self, node):
    """ Returns the heuristic cost estimate of a node """

    hcost = distance(self,self.start,node)
    
    return hcost


In [44]:
def calculate_fscore(self, node):
    """Calculate the f score of a node. """
    
    # F = G + H
    f_score = heuristic_cost_estimate(self, node) + get_gScore(self,node)
    
    return f_score
    

### Recording the best path

Now that you've implemented the various functions on scoring, you can record the best path to a given neighbor node from the current node!

In [45]:
def record_best_path_to(self, current, neighbor):
    """Record the best path to a node """
    
    # Records the best path to a node, by updating cameFrom, gScore, and fScore
    self.cameFrom[neighbor] = current
    self.gScore[neighbor] = self.get_tentative_gScore(current, neighbor)
    self.fScore[neighbor] = calculate_fscore(self,neighbor)
    
    


### Testing phase

In [46]:
# Associates implemented functions with PathPlanner class
PathPlanner.create_closedSet = create_closedSet
PathPlanner.create_openSet = create_openSet
PathPlanner.create_cameFrom = create_cameFrom
PathPlanner.create_gScore = create_gScore
PathPlanner.create_fScore = create_fScore
PathPlanner.set_map = set_map
PathPlanner.set_start = set_start
PathPlanner.set_goal = set_goal
PathPlanner.is_open_empty = is_open_empty
PathPlanner.get_current_node = get_current_node
PathPlanner.get_neighbors = get_neighbors
PathPlanner.get_gScore = get_gScore
PathPlanner.distance = distance
PathPlanner.get_tentative_gScore = get_tentative_gScore
PathPlanner.heuristic_cost_estimate = heuristic_cost_estimate
PathPlanner.calculate_fscore = calculate_fscore
PathPlanner.record_best_path_to = record_best_path_to

### Preliminary Test

The below is the first test case, just based off of one set of inputs.

In [53]:
planner = PathPlanner(map_40, 5, 34)
path = planner.path
if path == [5, 16, 37, 12, 34]:
    print("code works for these inputs!")
else:
    print("something is off, code produced the following:")
    print(path)

code works for these inputs!


#### Visualize

In [48]:
# Visualize the result
start = 5
goal = 34

show_map(map_40, start=start, goal=goal, path=PathPlanner(map_40, start, goal).path)

AttributeError: 'Graph' object has no attribute 'node'

### Testing your Code

In [51]:
from test import test

test(PathPlanner)

All tests pass!


---